In [1]:
import pandas as pd
import numpy as np
import re
import pickle
from datetime import datetime

In [2]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.io as pio
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.graph_objs import *

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,roc_curve, roc_auc_score, auc
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_val_score

In [4]:
# Define the color palette (17 colors).
Viridis= ['#440154', '#48186a', '#472d7b', '#424086', '#3b528b', '#33638d', '#2c728e', '#26828e', '#21918c', '#1fa088',
          '#28ae80', '#3fbc73', '#5ec962', '#84d44b', '#addc30','#d8e219', '#fde725']   

In [5]:
# read clean datafile
df = pd.read_csv('../data/dataset5.gz', compression='gzip', header=0, sep=',', quotechar='"')
print(df.shape)

(93701, 39)


With this many predictors, overfitting is definitely a risk. We'll use 3 steps to avoid overfit:
* standardize the predictors
* regularization
* kfold cross-validation

In [6]:
df.columns

Index(['device', 'failure', 'ndays', 'attribute2', 'attribute3', 'attribute4',
       'att5', 'att6', 'attribute7', 'S1F0', 'S1F1', 'W1F0', 'W1F1', 'Z1F0',
       'Z1F1', 'attribute2_lag01', 'attribute2_lag02', 'attribute2_lag03',
       'attribute2_lag04', 'attribute3_lag01', 'attribute3_lag02',
       'attribute3_lag03', 'attribute3_lag04', 'attribute4_lag01',
       'attribute4_lag02', 'attribute4_lag03', 'attribute4_lag04',
       'att5_lag01', 'att5_lag02', 'att5_lag03', 'att5_lag04', 'att6_lag01',
       'att6_lag02', 'att6_lag03', 'att6_lag04', 'attribute7_lag01',
       'attribute7_lag02', 'attribute7_lag03', 'attribute7_lag04'],
      dtype='object')

## Create a testing dataset

In [7]:
# Split the data
df_train, df_test = train_test_split(df, test_size = .3, random_state=42)

#### Dealing with imbalanced classes in the training dataset

In [8]:
from sklearn.utils import resample
# Separate majority and minority classes
df_majority = df_train[df_train['failure']==0]
df_minority = df_train[df_train['failure']==1]
 
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=len(df_majority),    # to match majority class
                                 random_state=42) # reproducible results
 
# Combine majority class with upsampled minority class
df_train_upsampled = pd.concat([df_majority, df_minority_upsampled])
 
# Display new class counts
df_train_upsampled['failure'].value_counts()
# hat tip: https://elitedatascience.com/imbalanced-classes

1    65524
0    65524
Name: failure, dtype: int64

In [9]:
# final x-values (features)
X_train=df_train_upsampled.drop(['failure', 'device'], axis=1)
X_test=df_test.drop(['failure', 'device'], axis=1)

In [10]:
# final y-values
y_train=df_train_upsampled['failure'].values
y1_test=df_test[['device','failure']] # Hold onto the device variable for later use, but remove it from the modeling data.
y_test=df_test['failure'].values

In [11]:
# confirm that lengths match
assert len(X_train)==len(y_train)
assert len(X_test)==len(y_test)

# Preliminary model exploration

### Naive Bayes Classifier

In [12]:
gnb = GaussianNB()
# Fit on the training data
gnb_model = gnb.fit(X_train, y_train)
# Predict on the testing data
predictions=gnb_model.predict(X_test)
probabilities = gnb_model.predict_proba(X_test)[:,1]
# Calculate the roc-auc score
auc_nb=metrics.roc_auc_score(y_test, predictions)
acc_nb = metrics.accuracy_score(y_test, predictions)
f1_nb = metrics.f1_score(y_test, predictions)
# Display
print('F1 Score', "%.4f" % round(f1_nb,4))
print('Accuracy', "%.4f" % round(acc_nb,4))
print('AUC Score', "%.4f" % round(auc_nb,4))

F1 Score 0.0045
Accuracy 0.7784
AUC Score 0.6693


### Logistic Regression

In [13]:
logreg = LogisticRegression()
# Fit on the training data
log_model=logreg.fit(X_train, y_train)
# Predict on the testing data
predictions=log_model.predict(X_test)
probabilities = log_model.predict_proba(X_test)[:,1]
# Calculate the roc-auc score
auc_log=metrics.roc_auc_score(y_test, predictions)
acc_log = metrics.accuracy_score(y_test, predictions)
f1_log = metrics.f1_score(y_test, predictions)
# Display
print('F1 Score', "%.4f" % round(f1_log,4))
print('Accuracy', "%.4f" % round(acc_log,4))
print('AUC Score', "%.4f" % round(auc_log,4))

F1 Score 0.0109
Accuracy 0.8901
AUC Score 0.7852


### KNN Classifier

In [14]:
knn = KNeighborsClassifier(n_neighbors=7)
# Fit on the training data
knn_model=knn.fit(X_train, y_train)
# Predict on the testing data
predictions=knn_model.predict(X_test)
probabilities = knn_model.predict_proba(X_test)[:,1]
# Calculate the roc-auc score
auc_knn=metrics.roc_auc_score(y_test, predictions)
acc_knn = metrics.accuracy_score(y_test, predictions)
f1_knn = metrics.f1_score(y_test, predictions)
# Display
print('F1 Score', "%.4f" % round(f1_knn,4))
print('Accuracy', "%.4f" % round(acc_knn,4))
print('AUC Score', "%.4f" % round(auc_knn,4))

F1 Score 0.0202
Accuracy 0.9965
AUC Score 0.5187


### Random Forest

In [15]:
rf = RandomForestClassifier()
# Fit on the training data
rf_model=rf.fit(X_train, y_train)
# Predict on the testing data
predictions=rf_model.predict(X_test)
probabilities = rf_model.predict_proba(X_test)[:,1]
# Calculate the roc-auc score
auc_rf=metrics.roc_auc_score(y_test, predictions)
acc_rf = metrics.accuracy_score(y_test, predictions)
f1_rf = metrics.f1_score(y_test, predictions)
# Display
print('F1 Score', "%.4f" % round(f1_rf,4))
print('Accuracy', "%.4f" % round(acc_rf,4))
print('AUC Score', "%.4f" % round(auc_rf,4))

F1 Score 0.0000
Accuracy 0.9988
AUC Score 0.4999


### Comparison of Four Models

In [16]:
f1=[f1_nb, f1_log, f1_knn, f1_rf]
acc=[acc_nb, acc_log, acc_knn, acc_rf]
auc=[auc_nb, auc_log, auc_knn, auc_rf]
models=['naive bayes', 'logistic regression', 'k-nearest neighbors', 'random forest']
index=['F1 score', 'Accuracy', 'AUC score']
results=pd.DataFrame([f1, acc, auc], index=index, columns=models)

In [17]:
results.head()

,naive bayes,logistic regression,k-nearest neighbors,random forest
F1 score,0.004476,0.010887,0.020202,0.000000
Accuracy,0.778450,0.890114,0.996549,0.998826
AUC score,0.669322,0.785151,0.518700,0.499858


In [18]:
# Pickle those results for comparison with tensorflow.
with open('model_metrics.pkl', 'wb') as f:
    pickle.dump(results, f)
f.close()

In [19]:
# Let's display that with plotly.
mydata1 = go.Bar(
    x=results.loc['F1 score'].index,
    y=results.loc['F1 score'],
    name=results.index[0],
    marker=dict(color=Viridis[16])
)
mydata2 = go.Bar(
    x=results.loc['Accuracy'].index,
    y=results.loc['Accuracy'],
    name=results.index[1],
    marker=dict(color=Viridis[10])
)
mydata3 = go.Bar(
    x=results.loc['AUC score'].index,
    y=results.loc['AUC score'],
    name=results.index[2],
    marker=dict(color=Viridis[0])
)
mylayout = go.Layout(
    title='Comparison of Possible Models',
    xaxis = dict(title = 'Predictive models'), # x-axis label
    yaxis = dict(title = 'Score'), # y-axis label
    
)
fig = go.Figure(data=[mydata1, mydata2, mydata3], layout=mylayout)
iplot(fig)

## Looks like we're overfitting

Try optimizing the  model to avoid overfit:
* gridsearch 
* kfold crossvalidation
* test on hold-out dataset

### Tuning the Logistic Regression model

In [20]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="darkgrid", color_codes=None)

In [21]:
# # Create regularization penalty space (l1=ridge, l2=lasso)
# penalty = ['l1', 'l2'] 

# # Create regularization hyperparameter space
# C = np.logspace(0, 4, 10)

# # Create hyperparameter options
# hyperparameters = dict(C=C, penalty=penalty)

# # Create grid search using 5-fold cross validation
# grid_lr = GridSearchCV(LogisticRegression(), hyperparameters, cv=5,  n_jobs = 1, verbose=0)
# grid_lr.fit(X_train, y_train)
# print(grid_lr.best_params_)

In [22]:
# # Instantiate the model using those parameters
# log_model = grid_lr.best_estimator_

# # Pickle the results of gridsearch so we don't have to do that again!
# with open('logistic_best_params.pkl', 'wb') as f:
#     pickle.dump(log_model, f)
# f.close()

In [23]:
# Unpickle the results of gridsearch
pickle_off = open('logistic_best_params.pkl','rb')
log_model = pickle.load(pickle_off)
print(log_model)

LogisticRegression(C=1291.5496650148827, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)


In [24]:
# Predict on the testing data
predictions=log_model.predict(X_test)
probabilities = log_model.predict_proba(X_test)[:,1]

In [25]:
# Full list of metrics
def model_metrics(y_test, predictions):
    '''
    Calculate 5 standard model metrics
    Return a dictionary with the metrics
    '''
    f1 = metrics.f1_score(y_test, predictions)
    accuracy = metrics.accuracy_score(y_test, predictions)
    error = 1 - accuracy
    precision = metrics.precision_score(y_test, predictions)
    recall = metrics.recall_score(y_test, predictions)
    rocauc =  metrics.roc_auc_score(y_test, predictions)
    return {'f1 score':f1, 'accuracy score': accuracy, 'error rate': error, 'precision score': precision, 'recall score': recall, 'ROC-AUC score': rocauc}

model_metrics(y_test, predictions)

{'f1 score': 0.010893944248638257,
 'accuracy score': 0.89018533670093558,
 'error rate': 0.10981466329906442,
 'precision score': 0.0054909560723514208,
 'recall score': 0.68000000000000005,
 'ROC-AUC score': 0.78518621377198616}

### ROC-AUC (Receiver Operating Characteristic - Area Under the Curve)

In [1]:
FPR, TPR, _ = roc_curve(y_test, predictions)
roc_score=round(100*roc_auc_score(y_test, predictions),1)
roc_score

NameError: name 'roc_curve' is not defined

In [27]:
# ROC-AUC figure

data = [
    {
      'x':FPR, 
      'y':TPR, 
      'type':'scatter',
      'mode': 'lines',
      'name': 'AUC: '+str(roc_score)
      },
     {'x':[0,1], 
      'y':[0,1], 
      'type':'scatter',
      'mode': 'lines',
      'name': 'Baseline Area: 50.0'}]

layout = go.Layout(
    title = 'Receiver Operating Characteristic - Area Under Curve',
    width=650,
    height=600, 
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [28]:
# A confusion matrix tells us our false positives and false negatives:
matrix=confusion_matrix(y_test, predictions)
cm=pd.DataFrame(matrix, columns=['predicted: +', 'predicted: -'], index=['ground truth: +', 'ground truth: -'])
cm=cm.reset_index(drop=False)
cm=cm.rename(columns={'index': f'n = {len(y_test)}'})

In [29]:
import plotly.figure_factory as ff
table = ff.create_table(cm)
iplot(table)

### Feature important (logistic reg coefficients)

In [30]:
# Feature importance
results=pd.DataFrame(list(zip(X_test.columns, logreg.coef_[0])), columns=['feature', 'coefficient'])
results=results.sort_values(by='coefficient', ascending=False)

In [31]:
big_coeffs=results.head(15)

In [32]:
big_coeffs

,feature,coefficient
3,attribute4,0.535093
21,attribute4_lag01,0.469254
13,attribute2_lag01,0.417921
1,attribute2,0.417046
22,attribute4_lag02,0.406024
14,attribute2_lag02,0.367693
23,attribute4_lag03,0.364203
33,attribute7_lag01,0.353935
6,attribute7,0.352449
15,attribute2_lag03,0.327986


In [33]:
# Let's display that with Plotly.
mydata = [go.Bar(
    x=big_coeffs['feature'],
    y=big_coeffs['coefficient'],
    marker=dict(color=Viridis[::-1])
)]

mylayout = go.Layout(
    title='Logistic Regression Results',
    xaxis = dict(title = 'Coefficients'), 
    yaxis = dict(title = 'Odds Ratio to Failure'), 

)
fig = go.Figure(data=mydata, layout=mylayout)
pio.write_image(fig, '../images/logistic.png')
iplot(fig)

#### Were these features highly correlated with the outcome?

In [34]:
cols=list(big_coeffs.feature)[:5]
cols.append('failure')
cols

['attribute4',
 'attribute4_lag01',
 'attribute2_lag01',
 'attribute2',
 'attribute4_lag02',
 'failure']

In [35]:
df[cols].corr()
# No, they were not.

,attribute4,attribute4_lag01,attribute2_lag01,attribute2,attribute4_lag02,failure
attribute4,1.000000,0.991320,0.209833,0.213120,0.983055,0.064157
attribute4_lag01,0.991320,1.000000,0.210570,0.210180,0.991663,0.056897
attribute2_lag01,0.209833,0.210570,1.000000,0.991008,0.207976,0.058904
attribute2,0.213120,0.210180,0.991008,1.000000,0.206883,0.059865
attribute4_lag02,0.983055,0.991663,0.207976,0.206883,1.000000,0.052150
failure,0.064157,0.056897,0.058904,0.059865,0.052150,1.000000


## How often are we accurate when aggregated to the device level?

In [36]:
y1_test=y1_test.reset_index(drop=True) # for the concat to work correctly, must have a clean index.
preds_df=pd.DataFrame(predictions, columns=['preds'])
combined_testdf=pd.concat([y1_test, preds_df], axis=1)

print('y_test: ', len(y1_test))
print('probabilities: ', len(preds_df))
print('combined: ', len(combined_testdf))

y_test:  28111
probabilities:  28111
combined:  28111


In [37]:
combined_testdf.head()

,device,failure,preds
0,S1F0S68M,0,0
1,Z1F1R76A,0,0
2,W1F16RA7,0,0
3,S1F131F6,0,0
4,W1F14GTK,0,0


In [38]:
aggdf=combined_testdf.groupby('device')[['device', 'failure', 'preds']].mean().reset_index(drop=False)
print(aggdf.shape)
aggdf.head()

(676, 3)


,device,failure,preds
0,S1F01E6Y,0.000000,0.000000
1,S1F01XDJ,0.000000,0.000000
2,S1F023H2,0.166667,0.833333
3,S1F02L38,0.000000,0.000000
4,S1F03YZM,0.012821,0.461538


In [39]:
aggdf['failed']=0
aggdf.loc[aggdf['failure']>0, 'failed']=1
aggdf['failed'].value_counts()

0    651
1     25
Name: failed, dtype: int64

In [40]:
aggdf['pred_failed']=0
aggdf.loc[aggdf['preds']>0, 'pred_failed']=1
aggdf['pred_failed'].value_counts()

0    536
1    140
Name: pred_failed, dtype: int64

In [41]:
# Confusion Matrix (device level)
pd.crosstab(aggdf['failed'], aggdf['pred_failed'],  margins=True)

pred_failed,0,1,All
failed,,,
0,528,123,651
1,8,17,25
All,536,140,676


The classification model does a poor job of identifying true negatives. It only identifies 13% of failures.